# Chatbot v1
The purpose of this project is to create a simple chatbot and become familiar with the frameworks used to build an interactive interface. There are no language models used here.


# Chatbot 101

## Building a Basic Chatbot
Streamlit will be the framework used to build the user interface.  Streamlist was chosen for this project because it handles the heavy lifting for html and css. Without having to build up the front end, I can focus on prototyping and  chatbot functionality.


## Containers

Below is a chatbot script that will repeat the users input. It uses 3 methods from the Streamlit framework to manage messages. 

* `st.chat_message` used to display containers with the user's input and the bot's response.
* `st.chat_input` a widget that allows user to enter input(prompt)
* `st.session_state` a list to store the chat history so it can be displayed in the containers; a dictionary is used in the example below with keys `role` (the author of the message) and `content` (the message itself)

In [76]:
%%writefile chatbot_echo.py

import streamlit as st

st.title('Echo Bot')

# Initialize chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Show previous messages
for message in st.session_state.messages:
    # Formatting defaults for containers are loaded depending on the 'speaker'
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# User input box ':=' checks if prompt is not empty, then assigns it to the variable prompt
if prompt := st.chat_input("Ready for your command..."):
    # Show user message
    with st.chat_message("user"):
        st.markdown(prompt) # Shows the message in a container
    # Add user message to history
    st.session_state.messages.append({"role": "user", "content": prompt})

    # Bot repeats the user input
    response = f"Echo: {prompt}"
    with st.chat_message("assistant"):
        st.markdown(response)
    # Add response to chat history
    st.session_state.messages.append({"role":"assistant", "content":response})


Writing chatbot_echo.py


## Building an Interactive Chatbot with Streaming
This is an interactive chatbot that responds to the user with pre-determined messages.  A delay is added to simulate the bot thinking.

In [1]:
%%writefile basicbot.py

import streamlit as st
import random
import time

# Streamed response emulator
def response_generator():
    response = random.choice(
        [
            "Hello there! How can I assist you today?",
            "Hi, human! Is there anything I can help you with?",
            "Do you need help?",
            "What can I do you for?",
            "Sorry we are busy servicing customers at this time."
        ]
    )
    for word in response.split():
        yield word + " "
        time.sleep(0.05)
        
st.title('Basic Chatbot')

# Initialize chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat messages from history on app rerun
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# Accept user input
if prompt := st.chat_input("What is up?"):
    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": prompt})
    # Display user message in chat message container
    with st.chat_message("user"):
        st.markdown(prompt)

    # Display assistant response in chat message container
    with st.chat_message("assistant"):
        response = st.write_stream(response_generator())
    # Add assistant response to chat history
    st.session_state.messages.append({"role": "assistant", "content": response})

Writing basicbot.py


# Adding LLMs to our Chatbot
Let's try using OpenAI for the chatbot responses when we pass the chat history as a list of messages.

### API Key
Before we can access an LLM, an API key is needed for authentication.  This is set up in the shell's configuration file beforehand and is not displayed here for security reasons.

Once the key is set as an environment variable, we can establish a client to handle all the communication between the notebook and the server.

In [1]:
from dotenv import load_dotenv
import os
from openai import OpenAI

# Step 1: Create the client (initializes API connection)
load_dotenv()  # Loads .env variables
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Step 2: Call the model to make sure it's working
completion = client.responses.create(
    model="gpt-4o",
    input="Write a single sentence about LLMs.",
)

# Step 3: Print the result
print(completion.output_text)


Large Language Models (LLMs) are advanced AI systems designed to understand and generate human-like text based on vast datasets.


In [2]:
%%writefile chatbot_LLM.py

from openai import OpenAI
import streamlit as st
import os

st.title('Chatbot_LLM')

# Set OpenAI API key from environment
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Set a default model
if "openai_model" not in st.session_state:
    st.session_state["openai_model"] = "gpt-3.5-turbo"

# Initialize chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Show previous messages from history
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# Accept user input
if prompt := st.chat_input("Waiting for your command..."):
    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": prompt})
    # Display user message in chat container
    with st.chat_message("user"):
        st.markdown(prompt)

    # Response in chat message container
    with st.chat_message("assistant"):
        stream = client.chat.completions.create(
            model = st.session_state["openai_model"],
            messages=[
                {"role": m["role"], "content": m["content"]}
                for m in st.session_state.messages
            ],
            stream=True,
        )
        response = st.write_stream(stream)
    # Add assistant response to chat history
    st.session_state.messages.append({"role": "assistant", "content": response})

Writing chatbot_LLM.py
